In [1]:
rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

## 问题1
编写一个程序, get_response(saying, response_rules)输入是一个字符串 + 我们定义的 rules，例如上边我们所写的 pattern， 输出是一个回答。

In [2]:
def is_pattern_segment(word):
    return word.startswith('?*') and all(a.isalpha() for a in word[2:])

In [3]:
def is_variable(word):
    return word.startswith('?') and all(a.isalpha() for a in word[1:])

In [4]:
def pat_to_dict(patterns):
    return {k:v for k,v in patterns}

def pat_to_dict_without_star(patterns):
    return {k.replace('?*','?'):v for k,v in patterns}

In [5]:
# 记录每个匹配符号和对应的内容
def pat_match(pattern,saying):
    if not pattern or not saying: return []
    if is_pattern_segment(pattern[0]):
        return [(pattern[0],saying[0])] + pat_match(pattern[1:],saying[1:])
    else:
        if pattern[0] != saying[0]: 
            return []
        else: 
            return pat_match(pattern[1:],saying[1:])


以最大匹配长度来匹配目标

In [6]:
def find_match_index(pattern,saying):
    #检查后边的文字,pattern_part第一个部分是 * 号
    match_index = -1  # -1 代表默认的完全匹配不上
    pat,rest = pattern[0],pattern[1:]
    #这里 0 判断*号后面已经没有文字，saying后边部分都当成 * 来配对
    if not rest: return 0

    for i, token in enumerate(saying):
        if rest[0] != token : 
            continue
        else:
            match_index = i
    return match_index

In [7]:
pattern = '?*X is cute'
saying = 'cat is cute'

a =  find_match_index(pattern.split(),saying.split())
a

1

In [8]:
def pat_match_with_seg(pattern,saying):
    #My dog and cat is cute --> ?*X is cute
    #I want to eat your apple -->  I want to eat your ?*X
    if not pattern or not saying: return []
    saying_adding = saying[0]
    #如果遇到匹配符号,检查后面的文字哪里有相同的部分，组合后面相同的文字得到需要匹配的项目
    if is_pattern_segment(pattern[0]):
        match_index = find_match_index(pattern,saying) 
        if match_index == 0:
            return [(pattern[0],' '.join(saying))]
        elif match_index > 0: #如果找到把它们拼接起来
            saying[0]  = ' '.join(saying[:match_index])
            del saying[1:match_index]
            return [(pattern[0],saying[0])] + pat_match_with_seg(pattern[1:],saying[1:])       
        else: #match_index = -1 
            return []
    else: #如果第一个没有遇到,后半部分继续匹配,
        if pattern[0] == saying[0]:
            return pat_match_with_seg(pattern[1:],saying[1:])
        else: #如果没有遇到匹配符号，并且已经发现文字不同就直接返回空集合
            return []


In [9]:
pattern = 'I want ?*X and ?*Y'
saying = 'I want an orange and an apple'

pattern_match =  pat_match_with_seg(pattern.split(),saying.split())
pattern_match 

[('?*X', 'an orange'), ('?*Y', 'an apple')]

In [10]:
def substitude(match_sentence,rule_match_dict):
    if not match_sentence: return []
    partial = [rule_match_dict.get(match_sentence[0],match_sentence[0])] 
    return partial + substitude(match_sentence[1:],rule_match_dict)

In [11]:
import random 
def get_response(saying,rules):
    rule_resp = ['No' ,'answer']
    for k, v in rules.items():
        pattern_match = pat_match_with_seg(k.split(),saying.split())
        pat_dict = pat_to_dict_without_star(pattern_match)
        if pat_dict:  # 如果匹配到格式
            rule_resp = random.choice(v)
            #检查回复是否含有需要替换的部分
            if any(is_variable(a) for a in rule_resp.split()):
                rule_resp = substitude(rule_resp.split(),pat_dict)
            else :
                pass
    return rule_resp


In [12]:

saying = 'I was an dinosaur'
# saying = 'Lily hello uuuu'

x = get_response(saying,rules)
x

['Were', 'you', 'really', 'an dinosaur', '?']

## 问题2 ：
改写以上程序，将程序变成能够支持中文输入的模式。 提示: 你可以需用用到 jieba 分词

In [ ]:
import re 
import string
import jieba
import random 

rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really ?y ?', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}


def is_variable(word):
    return word.startswith('?') and all(a.isalpha() for a in word[1:])

def is_pattern_segment(word):
    return word.startswith('?*') and all(a.isalpha() for a in word[2:])

def is_chinese(sentence):
    if ' ' not in sentence: return True
    else: return False

def pat_to_dict_without_star(patterns):
    return {k.replace('?*','?'):v for k,v in patterns.items()}

def split_pattern(pattern):
    alphabets = string.ascii_lowercase
    pat_1 = any('?*{}'.format(x) in pattern for x in alphabets) 
    pat_2 = any('?{}'.format(x) in pattern for x in alphabets) 
    for a in alphabets:
        if pat_1:
            pattern = pattern.replace('?*{}'.format(a),',?*{},'.format(a))
        elif pat_2:
            pattern = pattern.replace('?{}'.format(a),',?{},'.format(a))
        else: 
            return []
    pattern = pattern.split(',')
    pattern_list = [s for s in pattern if s]
    return pattern_list

#过滤pattern
alphabets = ['?*{}'.format(x) for x in string.ascii_lowercase] \
          + ['?{}'.format(x)  for x in string.ascii_lowercase]

#list.index()扩展版本,填入的是经过过滤的pattern_list
def get_multi_index(target_list,complete_list):

    def get_index(target_list,complete_list):
        if not target_list: return []  
        return [complete_list.index(target_list[0])] + get_index(target_list[1:],complete_list[1:])

    index = get_index(target_list,complete_list)
    index = [x + i for i,x in enumerate(index)]
    return index 

def get_word_indexes(pattern_list,saying):
    record_index = []
    word_list = [x for x in pattern_list if x not in alphabets]
    #如果全都是匹配符号，则word_list应该是这句话本身
    if not word_list: 
        word_list = [saying]

    word_index = get_multi_index(word_list,saying)
    for w_index,word in zip(word_index,word_list):
        end_index = w_index + len(word)
        record_index.append((w_index,end_index))
    return record_index

#将连续的单个index，变成一组一组区间组合
def combine_indexes(index_list):
    index_list.sort()
    if len(index_list) < 2:
        return []
    else: 
        return [(index_list[0],index_list[1])] + combine_indexes(index_list[1:])

def pat_match_with_seg(pattern,saying):
    #My dog and cat is cute --> ?*X is cute
    #I want to eat your apple -->  I want to eat your ?*X
    # hello I was an dinorsounr --> ?*x I was ?*y
    record_dict = {}
    pattern_list = split_pattern(pattern)
    if not all(p in saying for p in pattern_list if p not in alphabets): #一旦遇到不存在的情况就返回
        return record_dict
    else:  #需要匹配出 ?*:
        sign_list = [x for x in pattern_list if is_pattern_segment(x) ]
        #需要求出sign_index的所有区间,可以通过word_index间接求出
        word_index = get_word_indexes(pattern_list,saying)
        #处理最后一个规则：只有一个匹配符号 ?*x ，遇到也是返回{}空集合，不返回{'?*x':'整个句子'} 没有意义
        #(进行匹配有意义，返回无意义)
        if word_index == [(0,len(saying))]:
            return record_dict  
        spread_word_index = set([ x[0] for x in  word_index ] + [x[1] for x in  word_index])
        #防止单词没有在第一个开始出现，加入头和尾
        spread_word_index.update({0,len(saying)})
        spread_word_index = list(spread_word_index)
        #获取组合后的全部区间
        sign_word_index = combine_indexes(spread_word_index)
        #全部区间 - 单词区间 = 符号的区间
        sign_index = list(set(sign_word_index)-set(word_index))
        sign_index.sort()
        for s, i in zip(sign_list,sign_index):
            value = saying[i[0]:i[1]]
            record_dict.update({s:value})    
    return record_dict

def substitude(match_sentence,rule_match_dict):
    for k,v in rule_match_dict.items():
        match_sentence = match_sentence.replace(k,v)
    return match_sentence

def get_response(saying,rules):
    #设置默认的回复
    rule_resp = random.choice(rules['?*x'])
    for k,v in list(rules.items())[:-1]:
        pat_dict = pat_match_with_seg(k,saying)
        if pat_dict:  # 如果匹配到格式
            rule_resp = random.choice(v)
            #检查回复是否含有需要替换的部分
            if any(is_variable(a) for a in split_pattern(rule_resp)):
                pat_dict = pat_to_dict_without_star(pat_dict)
                rule_resp = substitude(rule_resp,pat_dict)
                return rule_resp
    return rule_resp

def convert_test_saying(rules):
    sign_list = ['?*x','?*y','?*z']
    replace_list = ['苹果','西瓜','菠萝'] 
    record_list = []
    for k in rules.keys():
        for s,r in zip(sign_list,replace_list):
            k = k.replace(s,r)
        record_list.append(k)
    return record_list

sayings = convert_test_saying(rule_responses)

for s in sayings:
    resp = get_response(s,rule_responses)
    print(s,'-->',resp)


## 问题3
多设计一些模式，让这个程序变得更好玩，多和大家交流，看看大家有什么好玩的模式

没写

## 问题4
这样的程序有什么优点？
--适用性比较强，可拓展模板

有什么缺点？
--不能匹配连续的两个*号，中文依赖jieba分词结果

你有什么可以改进的方法吗？
没有

什么是数据驱动？数据驱动在这个程序里如何体现？
--通过数据的格式去设计匹配模版

数据驱动与 AI 的关系是什么？
---算法建立在数据的基础上